In [1]:
import tensorflow as tf
import numpy as np
import small_data_generators as sdg
import copy

In [3]:
FC_SIZE = 1024
DTYPE = tf.float32


def _weight_variable(name, shape):
    return tf.get_variable(name, shape, tf.float32, tf.truncated_normal_initializer(stddev=0.1))


def _bias_variable(name, shape):
    return tf.get_variable(name, shape, DTYPE, tf.constant_initializer(0.1, dtype=DTYPE))


def inference(boxes, num_props,num_classes):
    prev_layer = boxes

    in_filters = num_props
    with tf.variable_scope('conv1') as scope:
        out_filters = 16
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name=scope.name)
        prev_layer = conv1
        in_filters = out_filters

    pool1 = tf.nn.max_pool3d(prev_layer, ksize=[1, 3, 3, 3, 1], strides=[1, 2, 2, 2, 1], padding='SAME')
    norm1 = tf.layers.batch_normalization(pool1)

    prev_layer = norm1

    with tf.variable_scope('conv2') as scope:
        out_filters = 32
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name=scope.name)

        prev_layer = conv2
        in_filters = out_filters

    # normalize prev_layer here
    pool2 = tf.nn.max_pool3d(prev_layer, ksize=[1, 3, 3, 3, 1], strides=[1, 2, 2, 2, 1], padding='SAME')
    norm2 = tf.layers.batch_normalization(pool2)
        
    with tf.variable_scope('conv3_1') as scope:
        out_filters = 64
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        prev_layer = tf.nn.relu(bias, name=scope.name)
        in_filters = out_filters

    with tf.variable_scope('conv3_2') as scope:
        out_filters = 64
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        prev_layer = tf.nn.relu(bias, name=scope.name)
        in_filters = out_filters

    with tf.variable_scope('conv3_3') as scope:
        out_filters = 32
        kernel = _weight_variable('weights', [5, 5, 5, in_filters, out_filters])
        conv = tf.nn.conv3d(prev_layer, kernel, [1, 1, 1, 1, 1], padding='SAME')
        biases = _bias_variable('biases', [out_filters])
        bias = tf.nn.bias_add(conv, biases)
        prev_layer = tf.nn.relu(bias, name=scope.name)
        in_filters = out_filters

    # normalize prev_layer here
    prev_layer = tf.layers.batch_normalization(prev_layer)
    prev_layer = tf.nn.max_pool3d(prev_layer, ksize=[1, 3, 3, 3, 1], strides=[1, 2, 2, 2, 1], padding='SAME')


    with tf.variable_scope('local3') as scope:
        dim = np.prod(prev_layer.get_shape().as_list()[1:])
        prev_layer_flat = tf.reshape(prev_layer, [-1, dim])
        weights = _weight_variable('weights', [dim, FC_SIZE])
        biases = _bias_variable('biases', [FC_SIZE])
        local3 = tf.nn.relu(tf.matmul(prev_layer_flat, weights) + biases, name=scope.name)

    prev_layer = local3

    with tf.variable_scope('local4') as scope:
        dim = np.prod(prev_layer.get_shape().as_list()[1:])
        prev_layer_flat = tf.reshape(prev_layer, [-1, dim])
        weights = _weight_variable('weights', [dim, FC_SIZE])
        biases = _bias_variable('biases', [FC_SIZE])
        local4 = tf.nn.relu(tf.matmul(prev_layer_flat, weights) + biases, name=scope.name)

    prev_layer = local4

    with tf.variable_scope('softmax_linear') as scope:
        dim = np.prod(prev_layer.get_shape().as_list()[1:])
        weights = _weight_variable('weights', [dim, num_classes])
        biases = _bias_variable('biases', [num_classes])
        softmax_linear = tf.add(tf.matmul(prev_layer, weights), biases, name=scope.name)

    return softmax_linear


def loss_function(logits, labels):
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(labels, tf.float32),logits=logits)
    return tf.reduce_mean(cross_entropy, name='xentropy_mean')

In [13]:
def apply_classification_loss_rnn(model_function):
    with tf.Graph().as_default() as g:
        num_channels = 4
        frame_depth = 16
        num_labels = 7
        with tf.device("/cpu:0"):
            trainer = tf.train.AdamOptimizer()
            x_ = tf.placeholder(tf.float32, [None,480, 720, frame_depth,1])
            y_ = tf.placeholder(tf.int32, [None,num_labels])
            y_logits = model_function(x_,1,num_labels)
            loss = loss_function(y_logits,y_)
            cross_entropy_loss = tf.reduce_mean(loss)
            train_op = trainer.minimize(cross_entropy_loss)
            y_prob = tf.sigmoid(y_logits)
            y_pred = tf.cast(tf.round(y_prob), tf.int32)
            correct_prediction = tf.equal(y_pred, y_)
            accuracy = tf.cast(correct_prediction, tf.float32)
    model_dict = {'graph': g, 'inputs': [x_, y_], 'train_op': train_op,
                  'pred':y_pred,'prob':y_prob,
                  'accuracy': accuracy, 'loss': cross_entropy_loss}
    return model_dict

In [14]:
apply_classification_loss_rnn(inference)

{'accuracy': <tf.Tensor 'Cast_2:0' shape=(?, 7) dtype=float32>,
 'graph': <tensorflow.python.framework.ops.Graph at 0x7f727f52b278>,
 'inputs': [<tf.Tensor 'Placeholder:0' shape=(?, 480, 720, 16, 1) dtype=float32>,
  <tf.Tensor 'Placeholder_1:0' shape=(?, 7) dtype=int32>],
 'loss': <tf.Tensor 'Mean:0' shape=() dtype=float32>,
 'pred': <tf.Tensor 'Cast_1:0' shape=(?, 7) dtype=int32>,
 'prob': <tf.Tensor 'Sigmoid:0' shape=(?, 7) dtype=float32>,
 'train_op': <tf.Operation 'Adam' type=NoOp>}

In [10]:
def train_model(model_dict, dataset_generators, epoch_n):
    with model_dict['graph'].as_default(), tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        train_record = ()
        test_record = ()
        for epoch_i in range(epoch_n):
            train_collect = []
            for iter_i, data_batch in enumerate(dataset_generators['train']):
                train_feed_dict = dict(zip(model_dict['inputs'], data_batch))
                train_to_compute = [model_dict['train_op'],model_dict['loss'],model_dict['accuracy']]
                sess.run(train_to_compute, feed_dict=train_feed_dict)
                train_collect.append(train_to_compute[1:])
            train_averages = np.mean(train_collect,axis=0)
            train_record += tuple(train_averages)

            collect_arr = []
            for a, test_batch in enumerate(dataset_generators['test']):
                test_feed_dict = dict(zip(model_dict['inputs'], test_batch))
                to_compute = [model_dict['loss'], model_dict['accuracy']]
                collect_arr.append(sess.run(to_compute, test_feed_dict))
            test_averages = np.mean(collect_arr, axis=0)
            test_record += tuple(test_averages)
            fmt = (epoch_i,) + tuple(averages)
            print('epoch {:d},  loss: {:.3f}, '
                  'accuracy: {:.3f}'.format(*fmt))
        print('the_train_record_is')
        print(train_record)
        print('the_train_record_is')
        print(test_record)

In [23]:
data_filelist = []
label_filelist = []

for i in range(10):
    data_filelist.append('array/X_seq_%d.npy'%(i+1))
    label_filelist.append('array/Y_seq_%d.npy'%(i+1))
print(data_filelist)
print(label_filelist)
    
    

dataset_generators = {
        'train': sdg.dataset_iterator_fbf(16,data_filelist, label_filelist,10),
        'test':  sdg.dataset_iterator_fbf(16,data_filelist, label_filelist,10),
    }

model_dict = apply_classification_loss_rnn(inference)
print(model_dict['graph'])
train_model(model_dict, dataset_generators, epoch_n = 2)

['array/X_seq_1.npy', 'array/X_seq_2.npy', 'array/X_seq_3.npy', 'array/X_seq_4.npy', 'array/X_seq_5.npy', 'array/X_seq_6.npy', 'array/X_seq_7.npy', 'array/X_seq_8.npy', 'array/X_seq_9.npy', 'array/X_seq_10.npy']
['array/Y_seq_1.npy', 'array/Y_seq_2.npy', 'array/Y_seq_3.npy', 'array/Y_seq_4.npy', 'array/Y_seq_5.npy', 'array/Y_seq_6.npy', 'array/Y_seq_7.npy', 'array/Y_seq_8.npy', 'array/Y_seq_9.npy', 'array/Y_seq_10.npy']


InvalidArgumentError: Cannot parse tensor from proto: dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 2764800
  }
  dim {
    size: 1024
  }
}
float_val: 0

	 [[Node: local3/weights/Adam/Initializer/zeros = Const[_class=["loc:@local3/weights"], dtype=DT_FLOAT, value=<Invalid TensorProto: dtype: DT_FLOAT tensor_shape { dim { size: 2764800 } dim { size: 1024 } } float_val: 0>, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'local3/weights/Adam/Initializer/zeros', defined at:
  File "/home/shen/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/shen/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/shen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/shen/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/sh